# Kode for datafortelling om operative mål i PO Pensjon
Målingene legges manuelt inn i et excel dokument som må lastes opp som "../data/Måleparametere PO Pensjon 2022.csv" ved oppdatering.

In [334]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict

import datastory as ds

sys.path.append('../lib')
import operative as opera
importlib.reload(opera)

<module 'operative' from '../lib/operative.py'>

In [335]:
df = pd.read_csv("../data/Måleparametere PO Pensjon 2022.csv", sep=";", header=2)

In [336]:
df["prosent_flagg"] = df.Nullpunktsmåling.apply(lambda x: "%" in x)

In [337]:
# Propager operative mål:
tmp = df["Operativt mål"].isna()
for i in range(df.shape[0]):
    if tmp[i]:
        df.iloc[i, 1] = df.iloc[i-1, 1]

In [338]:
def clean_numbers(x):
    if "%" in x:
        tmp = x[:-2]
        x = str(float(tmp)/100)
    if "-" in x:
        x = "0"
    return "".join(x.split(" "))

In [339]:
def set_plus(x):
    if x[0].isnumeric():
        return "+" + x
    else:
        return x

In [340]:
# Få alle tall på riktig format
maalinger_cols = ['Nullpunktsmåling', 'Måling T1'] # Legg til nye målinger her
for col in maalinger_cols:
    df[col] = df[col].apply(lambda x: clean_numbers(x)).astype("float")

prosent_endring_cols = ['% endring mot NP'] # Legg til ny endringsprosent hver tertial
for col in prosent_endring_cols:
    df[col] = df[col].apply(lambda x: set_plus(x))

In [341]:
df.head()

,Måleparameter,Operativt mål,NP gjennomført,Nullpunktsmåling,Måling T1,% endring mot NP,Måling T2,% endring mot NP.1,Måling T3,% endring mot NP.2,prosent_flagg
0,MP1: Antall saker i infotrygd vs. nytt system ...,1.1 - Lage ny løsning til Supplerende stønad ...,Ja,0.0,0.0,-,NaN,NaN,NaN,NaN,False
1,MP1: Antall kall fra Pesys til TPS for å hente...,1.2 - Fase ut bruk av TPS og lage nye løsninge...,Ja,67802.0,8930.0,-87 %,NaN,NaN,NaN,NaN,False
2,MP2: Antall kall fra Pesys til TPS,1.2 - Fase ut bruk av TPS og lage nye løsninge...,Ja,23939028.0,11126900.0,-54 %,NaN,NaN,NaN,NaN,False
3,MP3: Antall steder i Pesys der TPS kalles,1.2 - Fase ut bruk av TPS og lage nye løsninge...,Ja,526.0,448.0,-15 %,NaN,NaN,NaN,NaN,False
4,MP1: Antall brevmaler i gamle brevløsninger,1.3 - Fase ut gamle brevløsninger innen utgang...,Ja,257.0,257.0,+0 %,NaN,NaN,NaN,NaN,False


In [342]:
df.loc[df.shape[0]-1, "Nullpunktsmåling"] = np.nan
df.loc[df.shape[0]-1, "prosent_flagg"] = True
df.drop([0,7,11,17,18,19,28] , inplace=True)

In [343]:
df = df.T

In [344]:
df

,1,2,3,4,5,6,8,9,10,12,...,16,20,21,22,23,24,25,26,27,29
Måleparameter,MP1: Antall kall fra Pesys til TPS for å hente...,MP2: Antall kall fra Pesys til TPS,MP3: Antall steder i Pesys der TPS kalles,MP1: Antall brevmaler i gamle brevløsninger,Antall brevmaler i Doksys,Antall brevmaler i Exstream,MP3: Antall brev som produseres i gamle brevlø...,Antall brev som produseres i Doksys,Antall brev som produseres i Exstream,MP1: Antall produksjonssettinger av Pesys,...,MP5: Antall sikkerhetssårbarheter i Pesys,MP2: Andel søknader mottatt digitalt (av total...,MP1: %-andel av selvbetjeningstjenestene som t...,MP2: Bruk av ny & gammel selvbetjeningsløsning,MP2: Bruk av gammel selvbetjeningsløsning,MP2: Bruk av ny selvbetjeningsløsning,MP1: Automatiseringsgrad,MP2: Feil og mangelliste (redusere),MP3: Selvbetjeningsgrad,MP2: Andel søknader mottatt digitalt (av total...
Operativt mål,1.2 - Fase ut bruk av TPS og lage nye løsninge...,1.2 - Fase ut bruk av TPS og lage nye løsninge...,1.2 - Fase ut bruk av TPS og lage nye løsninge...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.3 - Fase ut gamle brevløsninger innen utgang...,1.5 - Modernisering av Pesys,...,1.5 - Modernisering av Pesys,2.2 - Nye løsninger for ytelser til etterlatte,1.6 - Erstatte utdaterte selvbetjeningsløsning...,1.6 - Erstatte utdaterte selvbetjeningsløsning...,1.6 - Erstatte utdaterte selvbetjeningsløsning...,1.6 - Erstatte utdaterte selvbetjeningsløsning...,4.1 - Løsninger og tjenester er velfungerende ...,4.1 - Løsninger og tjenester er velfungerende ...,4.1 - Løsninger og tjenester er velfungerende ...,5.2 - Forbedre og koordinere innholdet for yte...
NP gjennomført,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
Nullpunktsmåling,67802.0,23939028.0,526.0,257.0,51.0,206.0,48832.0,12862.0,35970.0,25.0,...,476.0,0.0,0.05,125900.0,0.03,0.97,0.5,150.0,0.82,NaN
Måling T1,8930.0,11126900.0,448.0,257.0,51.0,206.0,48132.0,16027.0,32105.0,21.0,...,70.0,0.81,0.05,42259.0,0.01,0.99,0.5,143.0,0.81,0.81
% endring mot NP,-87 %,-54 %,-15 %,+0 %,+0 %,+0 %,-1 %,+25 %,-11 %,-16 %,...,-85 %,+81 %,+0 %,-66 %,-81 %,+3 %,+0 %,-5 %,-1 %,+81 %
Måling T2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
% endring mot NP.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Måling T3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
% endring mot NP.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [345]:
# Liste med operative mål
operatives = []
prev_op = ""
for i in range(df.shape[1]):
    # Hver kolonne er ett målepunkt
    col = df.iloc[:,i]
    op = col["Operativt mål"]
    mparam = col["Måleparameter"]
    new_op = op != prev_op
    if new_op:
        print(op)
        # Append objekt for opprettet mål
        if prev_op != "":
            operatives.append(Op)
        # Lag objekt for neste operative mål
        Op = opera.Operative(op)
    prev_op = op
    # Oppretter målepunkts-objekt for kolonnen
    Op.MPs[mparam] = opera.MP.from_series(col=col, n=len(operatives)%7)
# Append siste operative mål
operatives.append(Op)

1.2 - Fase ut bruk av TPS og lage nye løsninger for personopplysninger
1.3 - Fase ut gamle brevløsninger innen utgangen av 2023
1.5 - Modernisering av Pesys
2.2 - Nye løsninger for ytelser til etterlatte
1.6 - Erstatte utdaterte selvbetjeningsløsninger med nye som tilfredsstiller lovpålagte krav 
4.1 - Løsninger og tjenester er velfungerende (BAU)
5.2 - Forbedre og koordinere innholdet for ytelser til etterlatte på nav.no og i søknadsdialogen


In [346]:
#operatives[1].show_figs()

In [347]:
story = ds.DataStory("Operative mål i PO Pensjon")
story.markdown("Oversikt over operative mål i PO Pensjon og tilhørende målepunkter. Enkelte målepunkter er ikke inkludert her. Prosent i plottene angir endring fra nullpunktsmåling.")
for op in operatives:
    story.header(op.name, level=1)
    print(op.name)
    for _, mp in op.MPs.items():
        story.header(mp.name, level=2)
        story.plotly(mp.fig.to_json())
        print(mp.name)

1.2 - Fase ut bruk av TPS og lage nye løsninger for personopplysninger
MP1: Antall kall fra Pesys til TPS for å hente samboerinformasjon

MP2: Antall kall fra Pesys til TPS
MP3: Antall steder i Pesys der TPS kalles
1.3 - Fase ut gamle brevløsninger innen utgangen av 2023
MP1: Antall brevmaler i gamle brevløsninger
Antall brevmaler i Doksys
Antall brevmaler i Exstream
MP3: Antall brev som produseres i gamle brevløsninger
Antall brev som produseres i Doksys
Antall brev som produseres i  Exstream
1.5 - Modernisering av Pesys
MP1: Antall produksjonssettinger av Pesys
MP2: Andel produksjonssettinger av Pesys på kveldstid
MP3: Antall kall som går via buss/ESB
MP4: Antall kodelinjer i Pesys 
MP5: Antall sikkerhetssårbarheter i Pesys
2.2 - Nye løsninger for ytelser til etterlatte
MP2: Andel søknader mottatt digitalt (av totalt antall søknader)
1.6 - Erstatte utdaterte selvbetjeningsløsninger med nye som tilfredsstiller lovpålagte krav 
MP1: %-andel av selvbetjeningstjenestene som tilbys via ny

In [348]:
#story.publish(url="https://nada.ekstern.dev.nav.no/api") #dev
story.publish(url="https://nada.intern.nav.no/api") #prod
#story.update(url="https://nada.intern.nav.no/api", token='ed32f304-8fc4-489f-b08d-c6b1b719f22e')

<IPython.core.display.Javascript object>

'https://data.intern.nav.no/story/draft/287452ef-4370-4edf-96f1-e1c4ec7349b6'